In [3]:
import pandas as pd 
import numpy as np 

def calculate_indicator(df):
    # 1 & 2: Simple moving average 
    df['SMA_5'] = df['close'].rolling(window=5, min_periods=1).mean()
    df['SMA_20'] = df['close'].rolling(window=20, min_periods=1).mean()

    # 3: Exponential moving average (EMA)
    df['EMA_10'] = df['close'].ewm(span=10, adjust=False).mean()

    # 4 & 5: Momentum indicator
    df['Momentum_3'] = df['close'].pct_change(periods=3)
    df['Momentum_10'] = df['close'].pct_change(periods=10)

    # 6: RSI 
    def calculate_rsi(series, periods=14):
        delta = series.diff()
        gain = (delta.where(delta>0, 0)).rolling(window=periods, min_periods=1).mean()
        loss = (delta.where(delta<0, 0)).rolling(window=periods, min_periods=1).mean()
        loss = loss.replace(0, 0.0001)  # Avoid deviding by 0 
        rs = gain/loss
        rsi = 100 - (100/ 1 + rs)
        return rsi 

    df['RSI_14'] = calculate_rsi(df['close'], periods=14)

    # 7 & 8: MACD
    exp12 = df['close'].ewm(span=12, adjust=False).mean()
    exp26 = df['close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp26  - exp12
    df['MACD_Signal'] = df['close'].ewm(span=9, adjust=False).mean()

    # 9, 10 & 11: Bollinger bands 
    df['BB_Middle'] = df['close'].rolling(window=20, min_periods=1).mean()
    bb_std = df['close'].rolling(window=20, min_periods=1).std()
    df['BB_Upper'] = df['BB_Middle'] + (bb_std * 2)
    df['BB_Lower'] = df['BB_Middle'] - (bb_std * 2)

    # 12: On balance volumn (OBV)
    df['OBV'] = (np.sign(df['close'].diff()) * df['volume']).fillna(0).cumsum()
    return df

def fill_nan_value(df):
    print("Filling NaN values...")
    indicator_columns = [
        'SMA_5', 'SMA_20', 'EMA_10', 'Momentum_3', 'Momentum_10',
        'RSI_14', 'MACD', 'MACD_Signal', 'BB_Middle', 'BB_Upper', 'BB_Lower', 'OBV'
    ]

    for col in indicator_columns:
        if col in df.columns:
            df[col] = df[col].fillna(method='ffill').fillna(method='bfill')

    df['Momentum_3'] = df['Momentum_3'].fillna(0)
    df['Momentum_10'] = df['Momentum_10'].fillna(0)
    return df

stocks = ['TCB', 'PDR', 'VIX']
for stock in stocks:
    df = pd.read_csv(f'../data/{stock}.csv')
    df = calculate_indicator(df)
    df = fill_nan_value(df)
    df.to_csv(f'../data/{stock}_with_indicators.csv', index=False)


Filling NaN values...
Filling NaN values...
Filling NaN values...


/tmp/ipykernel_31454/275694514.py:53: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_31454/275694514.py:53: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_31454/275694514.py:53: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df[col].fillna(method='ffill').fillna(method='bfill')
